####  Choose best RAG

In [4]:
import os
import pandas as pd

In [5]:
from FlagEmbedding import BGEM3FlagModel

cross_model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=False)

test = pd.read_excel('../explore/rag_test_three.xlsx') # read xlsx test file

def cross_score(model_answer, ans):
    score = cross_model.compute_score([ans, model_answer], max_passage_length=128, weights_for_different_modes=[1.0, 1.0, 1.0])['colbert']
    return score


Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 56654.26it/s]


In [23]:
# dicts = [
#     ('Embedding', ['GigaChat', 'GigaChat', 'GigaChat''GigaChat']), 
#     ('Vector store', ['Neo4j (hybrid search)', 'Neo4j', 'chroma', 'chroma']),
#     ('Text Splitting', ['Custom Text Splitter', 'Custom Text Splitter', 'LangChain Recursive Text Splitter', 'LangChain Recursive Text Splitter']),
#     ('Prompt', ['Prompts Var. 1', 'Prompts Var. 2', 'Prompts Var. 3', 'Prompts Var. 4'])
#     ]
import numpy as np
# dicts = [
#     ('GigaChat', 'GigaChat', 'GigaChat', 'GigaChat'), 
#     ('Neo4j (hybrid search)', 'Neo4j', 'chroma', 'chroma'),
#     ('Custom Text Splitter', 'Custom Text Splitter', 'LangChain Recursive Text Splitter', 'LangChain Recursive Text Splitter'),
#     ('Prompts Var. 1', 'Prompts Var. 2', 'Prompts Var. 3', 'Prompts Var. 4')
#     ]
records = [
    ('GigaChat', 'Neo4j (hybrid search)', 'Custom Text Splitter', 'Prompts Var. 1'), 
    ('GigaChat', 'Neo4j', 'Custom Text Splitter', 'Prompts Var. 2'),
    ('GigaChat', 'chroma', 'LangChain Recursive Text Splitter', 'Prompts Var. 3'),
    ('GigaChat', 'chroma', 'LangChain Recursive Text Splitter', 'Prompts Var. 4')
    ]
# print(np.ndarray(dicts))
columns = ['Embedding', 'Vector store', 'Text Splitting', 'Prompt']
index = ['rag_test_one', 'rag_test_two', 'rag_test_three', 'rag_test_vector']
rag_test_model_info = pd.DataFrame.from_records(records, columns=columns, index=index)
rag_test_model_info

,Embedding,Vector store,Text Splitting,Prompt
rag_test_one,GigaChat,Neo4j (hybrid search),Custom Text Splitter,Prompts Var. 1
rag_test_two,GigaChat,Neo4j,Custom Text Splitter,Prompts Var. 2
rag_test_three,GigaChat,chroma,LangChain Recursive Text Splitter,Prompts Var. 3
rag_test_vector,GigaChat,chroma,LangChain Recursive Text Splitter,Prompts Var. 4


In [17]:
explore = '../explore/'
answer_columns = ['answer', 'rag answer']



rag_test_filenames = ['rag_test_one', 'rag_test_two', 'rag_test_three', 'rag_test_vector']
rag_test_extension = '.xlsx'
rag_test_paths = map(lambda x: os.path.join(explore, x + rag_test_extension), rag_test_filenames)
rag_test_dfs = list(
    map(
        lambda x: (pd.read_excel(x, usecols=answer_columns)).dropna(),
        rag_test_paths
        )
    )

rag_test_dfs[0].head()

,answer,rag answer
0,"Согласно пункту 5.11 ПоЗ, при закупках товаров...",1. Допустимо ли при расчете НМЦ указывать орие...
1,Согласно СК-03.03.02.01 Проведение закупки у е...,Диапазон цен при осуществлении закупки у единс...
2,"пункт 17.1.8. в случае, когда на участие в кон...","1. Да, договор может быть заключен с участнико..."
3,"Согласно пункту 1.2.43 ПоЗ, участником закупки...","1. Коллективные участники, представленные физи..."
4,Датой подведения итогов закупки для способа за...,1. В документах не указана конкретная дата зак...


In [18]:
def count_row_score(row):
    return cross_score(ans = row[answer_columns[0]], model_answer = row[answer_columns[1]])

rag_test_scores = pd.DataFrame()
for rag_test_index, filename in enumerate(rag_test_filenames):
    rag_test_scores[filename] = (rag_test_dfs[rag_test_index].apply(count_row_score, axis=1))


In [32]:
rag_test_scores_df = rag_test_scores.mean(axis=0).to_frame('cross_score')
rag_test_df = rag_test_model_info.join(rag_test_scores_df)
rag_test_df = rag_test_df.reset_index()

In [22]:
# !pip3 install plotly
# !pip3 install kaleido

'/api/docs/query/1'

#### Create beautiful image table

In [45]:
import plotly.figure_factory as ff
import pandas as pd

fig = ff.create_table(rag_test_df, index = False)
fig.update_layout(
    autosize=False,
    width=1400,
    height=200,
)

fig.write_image("/home/viktor/Downloads/table_plotly.png", scale=2)